In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.callbacks import EarlyStopping

## Data

In [ ]:
x = np.linspace(0,100,1001)
y = np.sin(x)

In [ ]:
plt.plot(x,y);

In [ ]:
df = pd.DataFrame(data=y, index=x, columns=['Sine'])
df

## Train Test Split

In [ ]:
test_percent = 0.1
test_point = np.round(len(df) * test_percent)
test_ind = int(len(df) - test_point)
test_ind

In [ ]:
train = df.iloc[:test_ind]
test = df.iloc[test_ind:]

In [ ]:
train.plot();

In [ ]:
test.plot();

## Scale Data

In [ ]:
scaler = MinMaxScaler()

scaled_train = scaler.fit_transform(train)
scaled_test = scaler.transform(test)

## Time Series Generator

In [ ]:
length = 2
batch_size = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=batch_size)

In [ ]:
len(scaled_train)

In [ ]:
len(generator)

In [ ]:
X, y = generator[0]

In [ ]:
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

In [ ]:
length = 50
batch_size = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=batch_size)

validation_generator = TimeseriesGenerator(scaled_test, scaled_test, length=length, batch_size=batch_size)

In [ ]:
X, y = generator[0]

In [ ]:
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

## Create a Recurrent Neural Network and Training

In [ ]:
n_features = 1

In [ ]:
model = Sequential()

model.add(Input(shape=(length, n_features)))

model.add(SimpleRNN(units=50))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='sgd', loss='mse')

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model.fit(generator, epochs=20, validation_data=validation_generator, callbacks=[early_stop])

## Evaluation and Inverse Transformation

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot([['loss','val_loss']]);

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
predictions = scaler.inverse_transform(test_predictions)

In [ ]:
test

In [ ]:
test['RNN Predictions'] = predictions

In [ ]:
test.plot();

## LSTM

In [ ]:
model = Sequential()

model.add(Input(shape=(length, n_features)))

model.add(LSTM(units=50))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='sgd', loss='mse')

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

In [ ]:
model.fit(generator, epochs=20, validation_data=validation_generator, callbacks=[early_stop])

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
predictions = scaler.inverse_transform(test_predictions)
test['LSTM Predictions'] = predictions
test.plot(figsize=(12,8));

## Forecasting

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [ ]:
length = 50 # output length
batch_size = 1
generator = TimeseriesGenerator(scaled_data, scaled_data, length=length, batch_size=batch_size)

In [ ]:
model = Sequential()

model.add(Input(shape=(length, n_features)))

model.add(LSTM(units=50))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='sgd', loss='mse')

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

In [ ]:
model.fit(generator, epochs=20, validation_data=validation_generator, callbacks=[early_stop])

In [ ]:
forecast = []

first_eval_batch = scaled_data[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    forecast.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
forecast = scaler.inverse_transform(forecast)

In [ ]:
len(forecast)

In [ ]:
forecast_index = np.arange(100.1, 110.1, step=0.1)

In [ ]:
len(forecast_index)

In [ ]:
plt.plot(df.index, df['Sine'])
plt.plot(forecast_index, forecast);